<a href="https://colab.research.google.com/github/KendallScott/Capstone/blob/main/Data%20API/Data_Cleanup_Tennessee.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References:
https://geemap.org/notebooks/114_dynamic_world/


# Imports and Setup

In [1]:
!pip -q install --upgrade folium
!pip -q install geopandas
!pip -q install geojson
!pip -q install eeconvert

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 103.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 11.6 MB/s eta 0:00:00


In [2]:
# reminder that if you are installing libraries in a Google Colab instance you will be prompted to restart your kernal

try:
    import geemap, ee
except ModuleNotFoundError:
    if 'google.colab' in str(get_ipython()):
        print("package not found, installing w/ pip in Google Colab...")
        !pip install geemap
    else:
        print("package not found, installing w/ conda...")
        !conda install mamba -c conda-forge -y
        !mamba install geemap -c conda-forge -y
    import geemap, ee

In [3]:
pip install eemont

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.7/134.7 kB 3.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.7/224.7 kB 20.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for eemont: filename=eemont-0.3.6-py3-none-any.whl size=135742 sha256=70d6379eabcfd07aec70891fcfc392118bbf11a64d23c97c1b624a412a70fc16
  Stored in directory: /root/.cache/pip/wheels/c2/79/2c/4fed17c3d3b466bbf4fe5872eec11f189147043b01152a4f75
  Created wheel for ee_extra: filename=ee_extra-0.0.15-py3-none-any.whl size=236755 sha256=71987ac52827c45b5afced376cbc834845d324b0fd5617455962ee5eb750d6d6
  Stored in directory: /root/.cache/pip/wheels/29/96/0e/4e36b0dfd85e16867723df739294c0aa45a65b191adac4d959
Successfully built eemont ee_extra


In [4]:
# Standard imports
import requests
import json

import pandas as pd
import numpy as np
from PIL import Image

# Geospatial processing packages
import geopandas as gpd
import geojson

import shapely
from shapely.geometry import box

# Mapping and plotting libraries
import matplotlib.pyplot as plt
import matplotlib.colors as cl
import ee
import eeconvert as eec
import geemap
import folium
import eemont

import geopandas as gpd
from shapely.geometry import shape

### Mount Drive

In [5]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


### Authenticate Google Earth Engine
Make sure you have signed up for access to Google Earth Engine at https://signup.earthengine.google.com/#!/. Once your request has been approved, you should be able to access Google Earth Engine at https://code.earthengine.google.com/.

In [6]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=bcqwrJl76OSg-Jp7R-RRo6i6PFrgK1HLsjJeXtqSkPw&tc=B7DxcSuZJMOObg1ixdjH8CAMY1PRjCXVRHlCWXB35ng&cc=jPr-x93tPnGnazq6aQp0ghHyNiQJHxEvMwxZb6jpne0

The authorization workflow will generate a code, which you should paste in the box below.


KeyboardInterrupt: ignored

In [ ]:
# Read the GeoJSON file
mines = gpd.read_file('/content/drive/MyDrive/TN_Released_mines.geojson')

# Read data using GeoPandas
print("Data dimensions: {}".format(mines.shape))

In [ ]:


mines = mines.drop(columns=['permit_app'])
mines = mines.drop(columns=['permittee'])
mines = mines.drop(columns=['post_smcra'])
mines = mines.drop(columns=['reported_a'])
mines = mines.drop(columns=['state_trib'])
mines = mines.drop(columns=['shape_Leng'])
mines = mines.drop(columns=['steep_slop'])
mines = mines.drop(columns=['permit_a_2'])
mines = mines.drop(columns=['path'])
mines = mines.drop(columns=['comment'])
mines = mines.drop(columns=['coal_bed_n'])
mines = mines.drop(columns=['informatio'])
mines = mines.drop(columns=['layer'])

mines = mines.rename(columns={'id': 'shapeName'})

In [ ]:
mines_exploded= mines.explode()
mines_exploded

In [ ]:
df = pd.DataFrame()
years = ['2023-07', '2021-07', '2019-07', '2017-07']
i=0

i_date = years[0]+'-01' # Initial date of interest (inclusive)
f_date = years[0]+'-31' # Final date of interest (exclusive)

while i<len(mines_exploded):
      fil_mines = mines_exploded.iloc[[i]]
      region = eec.gdfToFc(fil_mines)
      dc = ee.ImageCollection('GOOGLE/DYNAMICWORLD/V1').filterBounds(region).filterDate(i_date, f_date) # Get dynamic world image collection by asset ID
      polygon = mines_exploded['geometry'].values[i]
      if polygon.geom_type == 'Polygon': # Extract the coordinates from the polygon,
        coordinates = polygon.exterior.coords[:]
        polygon = ee.Geometry.MultiPolygon(coordinates) # Create an ee.Geometry object from the coordinates
        max_error = 1 # Set the maxError parameter
        projected_polygon = polygon.transform('EPSG:4326', max_error) # Project the geometry coordinates
        ac = dc.getRegion(projected_polygon, scale=200 ).getInfo() #decrease scale to get more rows of data/granularity      df_constructor =pd.DataFrame(ac)

        df_constructor =pd.DataFrame(ac)
        df_constructor.columns = df_constructor.iloc[0]
        df_constructor = df_constructor.tail(-1)
        df_constructor['Company'] = mines_exploded['company'].values[i]
        df_constructor['mine_id'] = mines_exploded['shapeName'].values[i]
        df_constructor['orig_perm_id'] = mines_exploded['permit_id'].values[i]
        df_constructor['start_date'] = i_date
        df_constructor['end_date'] = f_date
        df = pd.concat([df,df_constructor])
        df =df.drop_duplicates()
        df = df[df['label'].notna()]

        print(mines_exploded['company'].values[i])

      elif polygon.geom_type == 'LineString':
        coordinates = polygon.coords[:]
        polygon = ee.Geometry.LineString(coordinates)
        ac = dc.getRegion(polygon, scale=200 ).getInfo() #decrease scale to get more rows of data/granularity      df_constructor =pd.DataFrame(ac)

        df_constructor =pd.DataFrame(ac)
        df_constructor.columns = df_constructor.iloc[0]
        df_constructor = df_constructor.tail(-1)
        df_constructor['Company'] = mines_exploded['company'].values[i]
        df_constructor['mine_id'] = mines_exploded['shapeName'].values[i]
        df_constructor['orig_perm_id'] = mines_exploded['permit_id'].values[i]
        df_constructor['start_date'] = i_date
        df_constructor['end_date'] = f_date
        df = pd.concat([df,df_constructor])

        df =df.drop_duplicates()
        df = df[df['label'].notna()]
        print(mines_exploded['company'].values[i])
      i+=1



file_name = 'ky_'+i_date+f_date
from google.colab import files
df.to_csv(file_name, encoding = 'utf-8-sig')
files.download(file_name)

In [ ]:

file_name = 'tn_'+i_date+f_date
from google.colab import files
df.to_csv(file_name, encoding = 'utf-8-sig')
files.download(file_name)

In [ ]:
#mapping demo in python https://colab.research.google.com/github/QuantEcon/quantecon-notebooks-datascience/blob/master/applications/maps.ipynb#scrollTo=SI5i7rboSGFL&uniqifier=2
# Read the downloaded file

kentucky = pd.read_csv('/content/drive/MyDrive/Mines_Data/ky_2023-07-012023-07-31')
kentucky